In [14]:
import numpy as np
from datetime import datetime

def next_run_time(config_file, run_time):
    
    dat = [np.array(line.split(' ')) for line in open(config_file)] 
    mm = [l[0] for l in dat]
    hh = [l[1] for l in dat]
    cmd = [l[2].rstrip() for l in dat] 
    
    run_time_strip = run_time.split(':')
    
    next_run = []
    for i in range(len(dat)):
        if(hh[i] == "*" and mm[i] == '*'):#runs every minute, every hour
            next_run.append( run_time_strip[0] + ":" +  run_time_strip[1] + " today " + cmd[i] )
        elif(hh[i] == "*"): #runs every hour at a certain time
            ref_time = run_time_strip[0] + ":" + mm[i]
            next_run.append(every_hr_job(ref_time, run_time, mm[i], cmd[i]))
        elif(mm[i] == "*"): #runs every minute of a given hour
            ref_time = hh[i] + ":" + "00"
            next_run.append(fixed_hr_job(ref_time, run_time, hh[i], cmd[i]))
        else:
            ref_time = hh[i] + ":" + mm[i]
            next_run.append(fixed_time_job(ref_time, run_time, cmd[i]))
    return next_run

#cron job runs at fixed time every day
def fixed_time_job(ref_time, run_time, cmd):
    
    tsecs = time_diff(ref_time, run_time)
    tsecs = time_diff(ref_time, run_time)
    
    next_run =''
    if(tsecs <= 0): #before run hour
        next_run = ref_time + " today " + cmd
    else: #runs tomorrow
         next_run = ref_time + " tomorrow " + cmd
    return next_run

#job runs at a fixed hour every day, throughout the hour
def fixed_hr_job(ref_time, run_time, hh, cmd):
    
    tsecs_l = time_diff(ref_time, run_time)
    new_hr = int(hh) + 1
    ref_time_upper = str(new_hr) + ":" + "00"
    tsecs_h = time_diff(ref_time_upper, run_time)
    
    next_run =''
    if(tsecs_l <= 0): #before run hour
        next_run = ref_time + " today " + cmd
    elif (tsecs_h >= 0): #after run hour
        next_run = ref_time + " tomorrow " + cmd
    else: #runs now
         next_run = ref_time.split(':')[0] + ":" + run_time.split(':')[1] + " today " + cmd
    return next_run

#job runs every hour of the day but only at a certain time (in mins) 
def every_hr_job(ref_time, run_time, mm, cmd):
    
    tsecs = time_diff(ref_time, run_time)
    next_run =''
    if(tsecs <= 0): #in current hr
        next_run = ref_time + " today " + cmd
    else: #next hour
        new_hr = int(run_time.split(':')[0]) + 1
        if(new_hr == 24): #rollover to next day: assuming times are system generated so no scope for human error in input
            new_time = '00' + ':' + mm
            next_run = new_time + " tomorrow " + cmd
        else: #(new_hr < 24):
            new_time = str(new_hr) + ':' + mm
            next_run = new_time + " today " + cmd 
#         else: #error
#             next_run = "error specifying time"
    return next_run


In [15]:
def time_diff(cron_time, run_time): #to include error trapping for >24h time
    FMT = '%H:%M:%S'
    cron_time += ':00'
    run_time += ':00'
    tdelta = datetime.strptime(run_time, FMT) - datetime.strptime(cron_time, FMT)
    return (tdelta.total_seconds() //60)

In [16]:
config_file = "1_config.txt"
run_time = "12:10"
nxt_run = next_run_time(config_file, run_time)
nxt_run

['1:30 tomorrow /bin/run_me_daily',
 '12:45 today /bin/run_me_hourly',
 '12:10 today /bin/run_me_every_minute',
 '19:00 today /bin/run_me_sixty_times']